# Importing the packages and data

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import scipy
from sklearn.metrics import r2_score, mean_squared_error

from sklearn.neighbors import NearestNeighbors
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold

In [3]:
import sys
sys.path.insert(1, '../sar_dirichlet')
import dirichlet_regression

In [4]:
from func_test import cos_similarity, create_features_matrices

In [5]:
import pickle

# With two features

In [6]:
n_features = 2
n_classes = 3

In [7]:
np.random.seed(21)

beta = np.array([[0.  , -1.6 , 1.],
                 [0.  , 1.8, -1.4],
                 [0.  , 1.4, -1.1 ]])

gamma_var = np.round(np.random.normal(size=(n_features)),2)

In [8]:
n_repeat = 100
list_n_samples = [50,200,1000]

In [9]:
cov_matrix = np.array([[1., 0.2], [0.2, 1.]])

# Estimation of the parameters

In [32]:
n_samples=200
rho=0.1

X,Z,W = create_features_matrices(n_samples,n_features,choice_W='random_distance',nneighbors=10,cov_mat=cov_matrix)
Z[:,0] = 1

list_n_i = np.random.randint(1, high=100, size=n_samples)
M = np.identity(n_samples) - rho*W
mu = dirichlet_regression.compute_mu_spatial(X, beta, M)

Y = np.array([np.random.multinomial(list_n_i[i],mu[i])/list_n_i[i] for i in range(n_samples)])
Y = (Y*(n_samples-1)+1/n_classes)/n_samples

In [21]:
reg_test = dirichlet_regression.dirichletRegressor(spatial=False)
reg_test.fit(X, Y, parametrization='alternative', Z=Z, fit_intercept=False, verbose=0)

In [22]:
%%time
reg_spatial = dirichlet_regression.dirichletRegressor(spatial=True, maxfun=5000)
reg_spatial.fit(X, Y, parametrization='alternative', Z=Z, W=W, fit_intercept=False, verbose=1)


CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH
Wall time: 579 ms


In [23]:
%%time
reg_spatial_ce = dirichlet_regression.dirichletRegressor(spatial=True)
reg_spatial_ce.fit(X, Y, parametrization='alternative', Z=Z, W=W, fit_intercept=False, verbose=1, loss='crossentropy')


Optimization terminated successfully.
Wall time: 230 ms


Test on another set of data (rerun the generation of X and W beforehand)

In [24]:
mu_pred = reg_spatial.pred(X,W)
print('R2:',r2_score(Y,mu_pred))
print('Cos similarity:',cos_similarity(Y, mu_pred))
print('RMSE:',mean_squared_error(Y,mu_pred,squared=False))

R2: 0.8562087106114226
Cos similarity: 0.9870651051589614
RMSE: 0.07171750525814836


In [25]:
mu_pred = reg_spatial_ce.pred(X,W)
print('R2:',r2_score(Y,mu_pred))
print('Cos similarity:',cos_similarity(Y, mu_pred))
print('RMSE:',mean_squared_error(Y,mu_pred,squared=False))

R2: 0.879731645067282
Cos similarity: 0.9892719244502415
RMSE: 0.06273201934441458


---

In [42]:
seed=0

i=0

n_samples = list_n_samples[i]

true_params = np.concatenate([beta.flatten(),gamma_var, [rho]])

solutions_spatial_temp, solutions_no_spatial_temp = [], []
solutions_ce_spatial_temp, solutions_ce_no_spatial_temp = [], []

list_n_i = np.random.randint(100, high=10000, size=n_samples)

for _ in range(10):
    np.random.seed(seed)

    X,Z,W = create_features_matrices(n_samples,n_features,choice_W='random_distance',nneighbors=10,cov_mat=cov_matrix)
    Z[:,0] = 1
    M = np.identity(n_samples) - rho*W

    #list_n_i = np.random.randint(100, high=10000, size=n_samples)
    M = np.identity(n_samples) - rho*W
    mu = dirichlet_regression.compute_mu_spatial(X, beta, M)

    try:
        #print(np.random.multinomial(list_n_i[i],mu[i])/list_n_i[i])
        Y = np.array([np.random.multinomial(list_n_i[i],mu[i])/list_n_i[i] for i in range(n_samples)])
        Y = (Y*(n_samples-1)+1/n_classes)/n_samples
        print(Y[i])

    except RuntimeError:
        print("Factor is exactly singular")
    except np.linalg.LinAlgError:
        print("Singular matrix")

    seed+=1

[0.01580756 0.00666667 0.97752577]
[0.01869416 0.00666667 0.97463918]
[0.36845361 0.19189003 0.43965636]
[0.01821306 0.00666667 0.97512027]
[0.19862543 0.03601375 0.76536082]
[0.14185567 0.01725086 0.84089347]
[0.29580756 0.08027491 0.62391753]
[0.01869416 0.00666667 0.97463918]
[0.21113402 0.03986254 0.74900344]
[0.2438488  0.05381443 0.70233677]


In [21]:
def estimating_parameters(rho, n_repeat=100, list_n_samples=[50,200,1000], cov_matrix=None):
    list_solutions_spatial, list_solutions_no_spatial = [], []
    list_solutions_ce_spatial, list_solutions_ce_no_spatial = [], []

    seed=0

    for i in range(len(list_n_samples)):
        n_samples = list_n_samples[i]

        true_params = np.concatenate([beta.flatten(),gamma_var, [rho]])

        solutions_spatial_temp, solutions_no_spatial_temp = [], []
        solutions_ce_spatial_temp, solutions_ce_no_spatial_temp = [], []
        
        list_n_i = np.random.randint(100, high=10000, size=n_samples)
        
        for _ in range(n_repeat):
            np.random.seed(seed)

            X,Z,W = create_features_matrices(n_samples,n_features,choice_W='random_distance',nneighbors=10,cov_mat=cov_matrix)
            Z[:,0] = 1
            M = np.identity(n_samples) - rho*W
            
            #list_n_i = np.random.randint(100, high=10000, size=n_samples)
            M = np.identity(n_samples) - rho*W
            mu = dirichlet_regression.compute_mu_spatial(X, beta, M)
            
            try:
                Y = np.array([np.random.multinomial(list_n_i[i],mu[i])/list_n_i[i] for i in range(n_samples)])
                Y = (Y*(n_samples-1)+1/n_classes)/n_samples

                reg_spatial = dirichlet_regression.dirichletRegressor(spatial=True, maxfun=5000)
                reg_spatial.fit(X, Y, parametrization='alternative', Z=Z, W=W, fit_intercept=False, verbose=0)
                solutions_spatial_temp.append(np.concatenate([reg_spatial.beta.flatten(),reg_spatial.gamma,[reg_spatial.rho]]))

                reg_no_spatial = dirichlet_regression.dirichletRegressor(spatial=False, maxfun=5000)
                reg_no_spatial.fit(X, Y, parametrization='alternative', Z=Z, fit_intercept=False, verbose=0)
                solutions_no_spatial_temp.append(np.concatenate([reg_no_spatial.beta.flatten(),reg_no_spatial.gamma]))

                reg_spatial_ce = dirichlet_regression.dirichletRegressor(spatial=True, maxfun=5000)
                reg_spatial_ce.fit(X, Y, loss='crossentropy', W=W, fit_intercept=False, verbose=0)
                solutions_ce_spatial_temp.append(np.concatenate([reg_spatial_ce.beta.flatten(),[reg_spatial_ce.rho]]))

                reg_no_spatial_ce = dirichlet_regression.dirichletRegressor(spatial=False, maxfun=5000)
                reg_no_spatial_ce.fit(X, Y, loss='crossentropy', fit_intercept=False, verbose=0)
                solutions_ce_no_spatial_temp.append(reg_no_spatial_ce.beta.flatten())

            except RuntimeError:
                print("Factor is exactly singular")
            except np.linalg.LinAlgError:
                print("Singular matrix")

            seed+=1
        list_solutions_spatial.append(solutions_spatial_temp)
        list_solutions_no_spatial.append(solutions_no_spatial_temp)
        list_solutions_ce_spatial.append(solutions_ce_spatial_temp)
        list_solutions_ce_no_spatial.append(solutions_ce_no_spatial_temp)
    return(list_solutions_spatial, list_solutions_no_spatial, list_solutions_ce_spatial, list_solutions_ce_no_spatial)

## rho=0.1

In [22]:
%%time
list_solutions_spatial, list_solutions_no_spatial, list_solutions_ce_spatial, list_solutions_ce_no_spatial = estimating_parameters(0.1, n_repeat=100, list_n_samples=[50,200,1000], cov_matrix=cov_matrix)

C:\Users\tnguyen001\AppData\Roaming\Python\Python38\site-packages\scipy\optimize\_numdiff.py:576: RuntimeWarning: invalid value encountered in subtract
  df = fun(x) - f0
C:\Users\tnguyen001\AppData\Roaming\Python\Python38\site-packages\scipy\optimize\_numdiff.py:576: RuntimeWarning: invalid value encountered in subtract
  df = fun(x) - f0
C:\Users\tnguyen001\AppData\Roaming\Python\Python38\site-packages\scipy\optimize\_numdiff.py:576: RuntimeWarning: invalid value encountered in subtract
  df = fun(x) - f0


Wall time: 33min 28s


In [29]:
true_params_effective = np.concatenate([beta[:,1:].flatten(),gamma_var, [rho]])
true_params_effective_no_spatial = np.concatenate([beta[:,1:].flatten(),gamma_var])

In [23]:
np.save('Data Dirichlet/multinomial_solutions_spatial_rho01.npy',list_solutions_spatial)
np.save('Data Dirichlet/multinomial_solutions_no_spatial_rho01.npy',list_solutions_no_spatial)
np.save('Data Dirichlet/multinomial_solutions_ce_spatial_rho01.npy',list_solutions_ce_spatial)
np.save('Data Dirichlet/multinomial_solutions_ce_no_spatial_rho01.npy',list_solutions_ce_no_spatial)

## rho=0.5

In [24]:
%%time
list_solutions_spatial, list_solutions_no_spatial, list_solutions_ce_spatial, list_solutions_ce_no_spatial = estimating_parameters(0.5, n_repeat=100, list_n_samples=[50,200,1000], cov_matrix=cov_matrix)

C:\Users\tnguyen001\AppData\Roaming\Python\Python38\site-packages\scipy\optimize\_numdiff.py:576: RuntimeWarning: invalid value encountered in subtract
  df = fun(x) - f0


Wall time: 49min 6s


In [26]:
np.save('Data Dirichlet/multinomial_solutions_spatial_rho05.npy',list_solutions_spatial)
np.save('Data Dirichlet/multinomial_solutions_no_spatial_rho05.npy',list_solutions_no_spatial)
np.save('Data Dirichlet/multinomial_solutions_ce_spatial_rho05.npy',list_solutions_ce_spatial)
np.save('Data Dirichlet/multinomial_solutions_ce_no_spatial_rho05.npy',list_solutions_ce_no_spatial)

## rho = 0.9

In [27]:
%%time
list_solutions_spatial, list_solutions_no_spatial, list_solutions_ce_spatial, list_solutions_ce_no_spatial = estimating_parameters(0.9, n_repeat=100, list_n_samples=[50,200,1000], cov_matrix=cov_matrix)

C:\Users\tnguyen001\AppData\Roaming\Python\Python38\site-packages\scipy\optimize\_numdiff.py:576: RuntimeWarning: invalid value encountered in subtract
  df = fun(x) - f0


Singular matrix
Singular matrix


../sar_dirichlet\dirichlet_regression.py:467: RuntimeWarning: overflow encountered in exp
  phi = np.exp(np.matmul(Z,gamma_var))
../sar_dirichlet\dirichlet_regression.py:87: RuntimeWarning: invalid value encountered in subtract
  sum_ll = np.sum(loggamma(phi) - sum_loggamma_phi_mu + sum_phi_mu_times_logY)


Wall time: 1h 8min 1s


In [28]:
np.save('Data Dirichlet/multinomial_solutions_spatial_rho09.npy',list_solutions_spatial)
np.save('Data Dirichlet/multinomial_solutions_no_spatial_rho09.npy',list_solutions_no_spatial)
np.save('Data Dirichlet/multinomial_solutions_ce_spatial_rho09.npy',list_solutions_ce_spatial)
np.save('Data Dirichlet/multinomial_solutions_ce_no_spatial_rho09.npy',list_solutions_ce_no_spatial)

ValueError: setting an array element with a sequence. The requested array has an inhomogeneous shape after 1 dimensions. The detected shape was (3,) + inhomogeneous part.

In [29]:
with open("Data Dirichlet/multinomial_solutions_spatial_rho09.npy", "wb") as fp:   #Pickling
    pickle.dump(list_solutions_spatial, fp)
    
with open("Data Dirichlet/multinomial_solutions_no_spatial_rho09.npy", "wb") as fp:   #Pickling
    pickle.dump(list_solutions_no_spatial, fp)
    
with open("Data Dirichlet/multinomial_solutions_ce_spatial_rho09.npy", "wb") as fp:   #Pickling
    pickle.dump(list_solutions_ce_spatial, fp)
    
with open("Data Dirichlet/multinomial_solutions_ce_no_spatial_rho09.npy", "wb") as fp:   #Pickling
    pickle.dump(list_solutions_ce_no_spatial, fp)

## rho=0

In [276]:
%%time
list_solutions_spatial, list_solutions_no_spatial, list_solutions_ce_spatial, list_solutions_ce_no_spatial = estimating_parameters(0., n_repeat=100, list_n_samples=[50,200,1000], cov_matrix=cov_matrix)

Wall time: 22min 27s


In [277]:
np.save('Data Dirichlet/multinomial_solutions_spatial_rho00.npy',list_solutions_spatial)
np.save('Data Dirichlet/multinomial_solutions_no_spatial_rho00.npy',list_solutions_no_spatial)
np.save('Data Dirichlet/multinomial_solutions_ce_spatial_rho00.npy',list_solutions_ce_spatial)
np.save('Data Dirichlet/multinomial_solutions_ce_no_spatial_rho00.npy',list_solutions_ce_no_spatial)

# Analysis of the results

In [10]:
from texttable import Texttable
import latextable

In [11]:
list_solutions_spatial_01 = np.load('Data Dirichlet/multinomial_solutions_spatial_rho01.npy')
list_solutions_no_spatial_01 = np.load('Data Dirichlet/multinomial_solutions_no_spatial_rho01.npy')
list_solutions_ce_spatial_01 = np.load('Data Dirichlet/multinomial_solutions_ce_spatial_rho01.npy')
list_solutions_ce_no_spatial_01 = np.load('Data Dirichlet/multinomial_solutions_ce_no_spatial_rho01.npy')

In [12]:
list_solutions_spatial_05 = np.load('Data Dirichlet/multinomial_solutions_spatial_rho05.npy', allow_pickle=True)
list_solutions_no_spatial_05 = np.load('Data Dirichlet/multinomial_solutions_no_spatial_rho05.npy', allow_pickle=True)
list_solutions_ce_spatial_05 = np.load('Data Dirichlet/multinomial_solutions_ce_spatial_rho05.npy', allow_pickle=True)
list_solutions_ce_no_spatial_05 = np.load('Data Dirichlet/multinomial_solutions_ce_no_spatial_rho05.npy', allow_pickle=True)

In [13]:
with open("Data Dirichlet/multinomial_solutions_spatial_rho09.npy", "rb") as fp:   # Unpickling
    list_solutions_spatial_09 = pickle.load(fp)
    
with open("Data Dirichlet/multinomial_solutions_no_spatial_rho09.npy", "rb") as fp:   # Unpickling
    list_solutions_no_spatial_09 = pickle.load(fp)
    
with open("Data Dirichlet/multinomial_solutions_ce_spatial_rho09.npy", "rb") as fp:   # Unpickling
    list_solutions_ce_spatial_09 = pickle.load(fp)
    
with open("Data Dirichlet/multinomial_solutions_ce_no_spatial_rho09.npy", "rb") as fp:   # Unpickling
    list_solutions_ce_no_spatial_09 = pickle.load(fp)

In [14]:
true_params_effective_01 = np.concatenate([beta[:,1:].flatten(),gamma_var, [0.1]])
true_params_effective_05 = np.concatenate([beta[:,1:].flatten(),gamma_var, [0.5]])
true_params_effective_09 = np.concatenate([beta[:,1:].flatten(),gamma_var, [0.9]])
true_params_effective_no_spatial = np.concatenate([beta[:,1:].flatten(),gamma_var])

In [15]:
list_solutions_spatial_00 = np.load('Data Dirichlet/dirichlet_solutions_spatial_rho00.npy')
list_solutions_no_spatial_00 = np.load('Data Dirichlet/dirichlet_solutions_no_spatial_rho00.npy')
list_solutions_ce_spatial_00 = np.load('Data Dirichlet/dirichlet_solutions_ce_spatial_rho00.npy', allow_pickle=True)
list_solutions_ce_no_spatial_00 = np.load('Data Dirichlet/dirichlet_solutions_ce_no_spatial_rho00.npy', allow_pickle=True)
true_params_effective_00 = np.concatenate([beta[:,1:].flatten(),gamma_var, [0.]])

### Results of maximum likelihood

In [35]:
table_params = Texttable()
table_params.add_row(["Parameter", "S n=50", "S n=200", "S n=1000",
                      "NS n=50", "NS n=200", "NS n=1000"])
                      
#param_names = ["$\\beta_{01}$", "", "$\\beta_{02}$", "", "$\\beta_{11}$", "", "$\\beta_{12}$", "",
#               "$\\beta_{21}$", "", "$\\beta_{22}$", "", "$\\gamma_0$", "", "$\\gamma_1$", "", "$\\rho$", ""]

param_names = ["\\multirow{2}{*}{$\\beta_{01}$}", "\\multirow{2}{*}{$\\beta_{02}$}", "\\multirow{2}{*}{$\\beta_{11}$}",
               "\\multirow{2}{*}{$\\beta_{12}$}", "\\multirow{2}{*}{$\\beta_{21}$}", "\\multirow{2}{*}{$\\beta_{22}$}",
               "\\multirow{2}{*}{$\\gamma_0$}", "\\multirow{2}{*}{$\\gamma_1$}", "\\multirow{2}{*}{$\\rho$}" ]
param_names_2 = [""]*len(param_names)

columns_1 = []
columns_2 = []

for ns in range(3):
    col_to_add = [str(np.round(np.mean(list_solutions_spatial_01[ns][:,i] - true_params_effective_01[i]),3))
                  +' ('+str(np.round(np.std(list_solutions_spatial_01[ns][:,i] - true_params_effective_01[i]),3))+')' for i in range(len(true_params_effective_01))]
    columns_1.append(col_to_add)
    col_to_add = ['['+  str(np.round(mean_squared_error(list_solutions_spatial_01[ns][:,i], [true_params_effective_01[i]]*100),3)) +']' for i in range(len(true_params_effective_01))]
    columns_2.append(col_to_add)
    
for ns in range(3):
    col_to_add = [str(np.round(np.mean(list_solutions_no_spatial_01[ns][:,i] - true_params_effective_no_spatial[i]),3))
                  +' ('+str(np.round(np.std(list_solutions_no_spatial_01[ns][:,i] - true_params_effective_no_spatial[i]),3))+')' for i in range(len(true_params_effective_no_spatial))] + ["/"]
    columns_1.append(col_to_add)
    col_to_add = [' ['+  str(np.round(mean_squared_error(list_solutions_no_spatial_01[ns][:,i], [true_params_effective_no_spatial[i]]*100),3)) +']' for i in range(len(true_params_effective_no_spatial))] + ["/"]
    columns_2.append(col_to_add)
        
columns_1.insert(0,param_names)
columns_2.insert(0,param_names_2)

tr_columns_1 = np.transpose(columns_1)
tr_columns_2 = np.transpose(columns_2)
for i in range(len(tr_columns_1)):
    table_params.add_row(tr_columns_1[i])
    table_params.add_row(tr_columns_2[i])

print(table_params.draw())

+-----------+-----------+----------+----------+----------+----------+----------+
| Parameter | S n=50    | S n=200  | S n=1000 | NS n=50  | NS n=200 | NS       |
|           |           |          |          |          |          | n=1000   |
+-----------+-----------+----------+----------+----------+----------+----------+
| \multirow | 0.209     | 0.061    | 0.027    | 0.083    | -0.077   | -0.124   |
| {2}{*}{$\ | (0.043)   | (0.012)  | (0.051)  | (0.05)   | (0.025)  | (0.012)  |
| beta_{01} |           |          |          |          |          |          |
| $}        |           |          |          |          |          |          |
+-----------+-----------+----------+----------+----------+----------+----------+
|           | [0.045]   | [0.004]  | [0.003]  |  [0.009] |  [0.007] |  [0.015] |
+-----------+-----------+----------+----------+----------+----------+----------+
| \multirow | -0.035    | -0.009   | 0.007    | 0.069    | 0.1      | 0.106    |
| {2}{*}{$\ | (0.021)   | (0

In [36]:
print(latextable.draw_latex(table_params, caption="Difference between the estimated parameters and the true parameters ($rho=0.1$) with multinomial generated data. The results are presented as the mean of the differences on the 100 iterations, the standard deviation within parenthesis, and the standard deviation within square brackets."))

\begin{table}
	\begin{center}
		\begin{tabular}{|l|l|l|l|l|l|l|}
			\hline
			 \\
			\hline
			Parameter & S n=50 & S n=200 & S n=1000 & NS n=50 & NS n=200 & NS n=1000 \\
			\hline
			\multirow{2}{*}{$\beta_{01}$} & 0.209 (0.043) & 0.061 (0.012) & 0.027 (0.051) & 0.083 (0.05) & -0.077 (0.025) & -0.124 (0.012) \\
			\hline
			 & [0.045] & [0.004] & [0.003] &  [0.009] &  [0.007] &  [0.015] \\
			\hline
			\multirow{2}{*}{$\beta_{02}$} & -0.035 (0.021) & -0.009 (0.006) & 0.007 (0.05) & 0.069 (0.036) & 0.1 (0.017) & 0.106 (0.007) \\
			\hline
			 & [0.002] & [0.0] & [0.003] &  [0.006] &  [0.01] &  [0.011] \\
			\hline
			\multirow{2}{*}{$\beta_{11}$} & -0.191 (0.041) & -0.056 (0.01) & -0.023 (0.047) & -0.207 (0.054) & -0.081 (0.021) & -0.04 (0.01) \\
			\hline
			 & [0.038] & [0.003] & [0.003] &  [0.046] &  [0.007] &  [0.002] \\
			\hline
			\multirow{2}{*}{$\beta_{12}$} & 0.102 (0.021) & 0.029 (0.008) & 0.007 (0.008) & 0.113 (0.027) & 0.038 (0.013) & 0.018 (0.005) \\
			\hline
			 & [0.01

In [37]:
table_params = Texttable()
table_params.add_row(["Parameter", "S n=50", "S n=200", "S n=1000",
                      "NS n=50", "NS n=200", "NS n=1000"])
                      
#param_names = ["$\\beta_{01}$", "", "$\\beta_{02}$", "", "$\\beta_{11}$", "", "$\\beta_{12}$", "",
#               "$\\beta_{21}$", "", "$\\beta_{22}$", "", "$\\gamma_0$", "", "$\\gamma_1$", "", "$\\rho$", ""]

param_names = ["\\multirow{2}{*}{$\\beta_{01}$}", "\\multirow{2}{*}{$\\beta_{02}$}", "\\multirow{2}{*}{$\\beta_{11}$}",
               "\\multirow{2}{*}{$\\beta_{12}$}", "\\multirow{2}{*}{$\\beta_{21}$}", "\\multirow{2}{*}{$\\beta_{22}$}",
               "\\multirow{2}{*}{$\\gamma_0$}", "\\multirow{2}{*}{$\\gamma_1$}", "\\multirow{2}{*}{$\\rho$}" ]
param_names_2 = [""]*len(param_names)

columns_1 = []
columns_2 = []

for ns in range(3):
    col_to_add = [str(np.round(np.mean(np.array(list_solutions_spatial_05[ns])[:,i] - true_params_effective_05[i]),3))
                  +' ('+str(np.round(np.std(np.array(list_solutions_spatial_05[ns])[:,i] - true_params_effective_05[i]),3))+')' for i in range(len(true_params_effective_05))]
    columns_1.append(col_to_add)
    col_to_add = ['['+  str(np.round(mean_squared_error(np.array(list_solutions_spatial_05[ns])[:,i], [true_params_effective_05[i]]*len(list_solutions_spatial_05[ns])),3)) +']' for i in range(len(true_params_effective_05))]
    columns_2.append(col_to_add)
    
for ns in range(3):
    col_to_add = [str(np.round(np.mean(np.array(list_solutions_no_spatial_05[ns])[:,i] - true_params_effective_no_spatial[i]),3))
                  +' ('+str(np.round(np.std(np.array(list_solutions_no_spatial_05[ns])[:,i] - true_params_effective_no_spatial[i]),3))+')' for i in range(len(true_params_effective_no_spatial))] + ["/"]
    columns_1.append(col_to_add)
    col_to_add = [' ['+  str(np.round(mean_squared_error(np.array(list_solutions_no_spatial_05[ns])[:,i], [true_params_effective_no_spatial[i]]*len(list_solutions_no_spatial_05[ns])),3)) +']' for i in range(len(true_params_effective_no_spatial))] + ["/"]
    columns_2.append(col_to_add)
        
columns_1.insert(0,param_names)
columns_2.insert(0,param_names_2)

tr_columns_1 = np.transpose(columns_1)
tr_columns_2 = np.transpose(columns_2)
for i in range(len(tr_columns_1)):
    table_params.add_row(tr_columns_1[i])
    table_params.add_row(tr_columns_2[i])

print(table_params.draw())

+-----------+-----------+----------+----------+----------+----------+----------+
| Parameter | S n=50    | S n=200  | S n=1000 | NS n=50  | NS n=200 | NS       |
|           |           |          |          |          |          | n=1000   |
+-----------+-----------+----------+----------+----------+----------+----------+
| \multirow | 0.37      | 0.11     | 0.044    | 0.324    | 0.318    | 0.273    |
| {2}{*}{$\ | (0.156)   | (0.04)   | (0.101)  | (0.341)  | (0.208)  | (0.103)  |
| beta_{01} |           |          |          |          |          |          |
| $}        |           |          |          |          |          |          |
+-----------+-----------+----------+----------+----------+----------+----------+
|           | [0.161]   | [0.014]  | [0.012]  |  [0.222] |  [0.145] |  [0.085] |
+-----------+-----------+----------+----------+----------+----------+----------+
| \multirow | -0.052    | -0.012   | -0.001   | 0.617    | 0.633    | 0.608    |
| {2}{*}{$\ | (0.051)   | (0

In [38]:
print(latextable.draw_latex(table_params, caption="Difference between the estimated parameters and the true parameters ($rho=0.5$) with multinomial generated data. The results are presented as the mean of the differences on the 100 iterations, the standard deviation within parenthesis, and the standard deviation within square brackets."))

\begin{table}
	\begin{center}
		\begin{tabular}{|l|l|l|l|l|l|l|}
			\hline
			 \\
			\hline
			Parameter & S n=50 & S n=200 & S n=1000 & NS n=50 & NS n=200 & NS n=1000 \\
			\hline
			\multirow{2}{*}{$\beta_{01}$} & 0.37 (0.156) & 0.11 (0.04) & 0.044 (0.101) & 0.324 (0.341) & 0.318 (0.208) & 0.273 (0.103) \\
			\hline
			 & [0.161] & [0.014] & [0.012] &  [0.222] &  [0.145] &  [0.085] \\
			\hline
			\multirow{2}{*}{$\beta_{02}$} & -0.052 (0.051) & -0.012 (0.059) & -0.001 (0.025) & 0.617 (0.258) & 0.633 (0.138) & 0.608 (0.061) \\
			\hline
			 & [0.005] & [0.004] & [0.001] &  [0.448] &  [0.42] &  [0.373] \\
			\hline
			\multirow{2}{*}{$\beta_{11}$} & -0.416 (0.221) & -0.117 (0.06) & -0.048 (0.118) & -1.032 (0.311) & -1.004 (0.139) & -0.961 (0.07) \\
			\hline
			 & [0.222] & [0.017] & [0.016] &  [1.162] &  [1.028] &  [0.928] \\
			\hline
			\multirow{2}{*}{$\beta_{12}$} & 0.155 (0.045) & 0.045 (0.017) & 0.014 (0.027) & 0.406 (0.14) & 0.426 (0.078) & 0.43 (0.042) \\
			\hline
			 & [0.0

In [39]:
table_params = Texttable()
table_params.add_row(["Parameter", "S n=50", "S n=200", "S n=1000",
                      "NS n=50", "NS n=200", "NS n=1000"])
                      
#param_names = ["$\\beta_{01}$", "", "$\\beta_{02}$", "", "$\\beta_{11}$", "", "$\\beta_{12}$", "",
#               "$\\beta_{21}$", "", "$\\beta_{22}$", "", "$\\gamma_0$", "", "$\\gamma_1$", "", "$\\rho$", ""]

param_names = ["\\multirow{2}{*}{$\\beta_{01}$}", "\\multirow{2}{*}{$\\beta_{02}$}", "\\multirow{2}{*}{$\\beta_{11}$}",
               "\\multirow{2}{*}{$\\beta_{12}$}", "\\multirow{2}{*}{$\\beta_{21}$}", "\\multirow{2}{*}{$\\beta_{22}$}",
               "\\multirow{2}{*}{$\\gamma_0$}", "\\multirow{2}{*}{$\\gamma_1$}", "\\multirow{2}{*}{$\\rho$}" ]
param_names_2 = [""]*len(param_names)

columns_1 = []
columns_2 = []

for ns in range(3):
    col_to_add = [str(np.round(np.mean(np.array(list_solutions_spatial_09[ns])[:,i] - true_params_effective_09[i]),3))
                  +' ('+str(np.round(np.std(np.array(list_solutions_spatial_09[ns])[:,i] - true_params_effective_09[i]),3))+')' for i in range(len(true_params_effective_09))]
    columns_1.append(col_to_add)
    col_to_add = ['['+  str(np.round(mean_squared_error(np.array(list_solutions_spatial_09[ns])[:,i], [true_params_effective_09[i]]*len(list_solutions_spatial_09[ns])),3)) +']' for i in range(len(true_params_effective_09))]
    columns_2.append(col_to_add)
    
for ns in range(3):
    col_to_add = [str(np.round(np.mean(np.array(list_solutions_no_spatial_09[ns])[:,i] - true_params_effective_no_spatial[i]),3))
                  +' ('+str(np.round(np.std(np.array(list_solutions_no_spatial_09[ns])[:,i] - true_params_effective_no_spatial[i]),3))+')' for i in range(len(true_params_effective_no_spatial))] + ["/"]
    columns_1.append(col_to_add)
    col_to_add = [' ['+  str(np.round(mean_squared_error(np.array(list_solutions_no_spatial_09[ns])[:,i], [true_params_effective_no_spatial[i]]*len(list_solutions_no_spatial_09[ns])),3)) +']' for i in range(len(true_params_effective_no_spatial))] + ["/"]
    columns_2.append(col_to_add)
        
columns_1.insert(0,param_names)
columns_2.insert(0,param_names_2)

tr_columns_1 = np.transpose(columns_1)
tr_columns_2 = np.transpose(columns_2)
for i in range(len(tr_columns_1)):
    table_params.add_row(tr_columns_1[i])
    table_params.add_row(tr_columns_2[i])

print(table_params.draw())

+-----------+-----------+----------+----------+----------+----------+----------+
| Parameter | S n=50    | S n=200  | S n=1000 | NS n=50  | NS n=200 | NS       |
|           |           |          |          |          |          | n=1000   |
+-----------+-----------+----------+----------+----------+----------+----------+
| \multirow | 1.625     | 1.603    | 1.584    | 1.453    | 1.408    | 1.403    |
| {2}{*}{$\ | (0.092)   | (0.071)  | (0.048)  | (0.15)   | (0.074)  | (0.038)  |
| beta_{01} |           |          |          |          |          |          |
| $}        |           |          |          |          |          |          |
+-----------+-----------+----------+----------+----------+----------+----------+
|           | [2.649]   | [2.575]  | [2.512]  |  [2.135] |  [1.988] |  [1.97]  |
+-----------+-----------+----------+----------+----------+----------+----------+
| \multirow | 1.271     | -0.144   | -0.392   | 3.193    | 3.646    | 3.144    |
| {2}{*}{$\ | (2.669)   | (0

In [40]:
print(latextable.draw_latex(table_params, caption="Difference between the estimated parameters and the true parameters ($rho=0.9$) with multinomial generated data. The results are presented as the mean of the differences on the 100 iterations, the standard deviation within parenthesis, and the standard deviation within square brackets."))

\begin{table}
	\begin{center}
		\begin{tabular}{|l|l|l|l|l|l|l|}
			\hline
			 \\
			\hline
			Parameter & S n=50 & S n=200 & S n=1000 & NS n=50 & NS n=200 & NS n=1000 \\
			\hline
			\multirow{2}{*}{$\beta_{01}$} & 1.625 (0.092) & 1.603 (0.071) & 1.584 (0.048) & 1.453 (0.15) & 1.408 (0.074) & 1.403 (0.038) \\
			\hline
			 & [2.649] & [2.575] & [2.512] &  [2.135] &  [1.988] &  [1.97] \\
			\hline
			\multirow{2}{*}{$\beta_{02}$} & 1.271 (2.669) & -0.144 (0.663) & -0.392 (0.399) & 3.193 (1.076) & 3.646 (1.208) & 3.144 (0.803) \\
			\hline
			 & [8.738] & [0.46] & [0.313] &  [11.351] &  [14.754] &  [10.528] \\
			\hline
			\multirow{2}{*}{$\beta_{11}$} & -2.087 (0.35) & -2.208 (0.289) & -2.11 (0.267) & -1.971 (0.205) & -1.985 (0.105) & -1.95 (0.038) \\
			\hline
			 & [4.48] & [4.96] & [4.523] &  [3.926] &  [3.95] &  [3.803] \\
			\hline
			\multirow{2}{*}{$\beta_{12}$} & 1.016 (0.395) & 0.957 (0.293) & 1.052 (0.262) & 1.147 (0.267) & 1.177 (0.103) & 1.212 (0.035) \\
			\hline
			 & [1.

In [41]:
table_params = Texttable()
table_params.add_row(["Parameter", "S n=50", "S n=200", "S n=1000",
                      "NS n=50", "NS n=200", "NS n=1000"])

param_names = ["\\multirow{2}{*}{$\\beta_{01}$}", "\\multirow{2}{*}{$\\beta_{02}$}", "\\multirow{2}{*}{$\\beta_{11}$}",
               "\\multirow{2}{*}{$\\beta_{12}$}", "\\multirow{2}{*}{$\\beta_{21}$}", "\\multirow{2}{*}{$\\beta_{22}$}",
               "\\multirow{2}{*}{$\\gamma_0$}", "\\multirow{2}{*}{$\\gamma_1$}", "\\multirow{2}{*}{$\\rho$}" ]
param_names_2 = [""]*len(param_names)

columns_1 = []
columns_2 = []

for ns in range(3):
    col_to_add = [str(np.round(np.mean(list_solutions_spatial_00[ns][:,i] - true_params_effective_00[i]),3))
                  +' ('+str(np.round(np.std(list_solutions_spatial_00[ns][:,i] - true_params_effective_00[i]),3))+')' for i in range(len(true_params_effective_00))]
    columns_1.append(col_to_add)
    col_to_add = ['['+  str(np.round(mean_squared_error(list_solutions_spatial_00[ns][:,i], [true_params_effective_00[i]]*100),3)) +']' for i in range(len(true_params_effective_00))]
    columns_2.append(col_to_add)
    
for ns in range(3):
    col_to_add = [str(np.round(np.mean(list_solutions_no_spatial_00[ns][:,i] - true_params_effective_no_spatial[i]),3))
                  +' ('+str(np.round(np.std(list_solutions_no_spatial_00[ns][:,i] - true_params_effective_no_spatial[i]),3))+')' for i in range(len(true_params_effective_no_spatial))] + ["/"]
    columns_1.append(col_to_add)
    col_to_add = [' ['+  str(np.round(mean_squared_error(list_solutions_no_spatial_00[ns][:,i], [true_params_effective_no_spatial[i]]*100),3)) +']' for i in range(len(true_params_effective_no_spatial))] + ["/"]
    columns_2.append(col_to_add)
        
columns_1.insert(0,param_names)
columns_2.insert(0,param_names_2)

tr_columns_1 = np.transpose(columns_1)
tr_columns_2 = np.transpose(columns_2)
for i in range(len(tr_columns_1)):
    table_params.add_row(tr_columns_1[i])
    table_params.add_row(tr_columns_2[i])

print(table_params.draw())

NameError: name 'true_params_effective_00' is not defined

### Results on cross-entropy

In [16]:
true_params_effective_01_ce = np.concatenate([beta[:,1:].flatten(), [0.1]])
true_params_effective_05_ce = np.concatenate([beta[:,1:].flatten(), [0.5]])
true_params_effective_09_ce = np.concatenate([beta[:,1:].flatten(), [0.9]])
true_params_effective_no_spatial_ce = np.concatenate([beta[:,1:].flatten()])

In [17]:
table_params = Texttable()
table_params.add_row(["Parameter", "S n=50", "S n=200", "S n=1000",
                      "NS n=50", "NS n=200", "NS n=1000"])
                      
#param_names = ["$\\beta_{01}$", "", "$\\beta_{02}$", "", "$\\beta_{11}$", "", "$\\beta_{12}$", "",
#               "$\\beta_{21}$", "", "$\\beta_{22}$", "", "$\\gamma_0$", "", "$\\gamma_1$", "", "$\\rho$", ""]

param_names = ["\\multirow{2}{*}{$\\beta_{01}$}", "\\multirow{2}{*}{$\\beta_{02}$}", "\\multirow{2}{*}{$\\beta_{11}$}",
               "\\multirow{2}{*}{$\\beta_{12}$}", "\\multirow{2}{*}{$\\beta_{21}$}", "\\multirow{2}{*}{$\\beta_{22}$}",
               "\\multirow{2}{*}{$\\rho$}" ]
param_names_2 = [""]*len(param_names)

columns_1 = []
columns_2 = []

for ns in range(3):
    col_to_add = [str(np.round(np.mean(list_solutions_ce_spatial_01[ns][:,i] - true_params_effective_01_ce[i]),3))
                  +' ('+str(np.round(np.std(list_solutions_ce_spatial_01[ns][:,i] - true_params_effective_01_ce[i]),3))+')' for i in range(len(true_params_effective_01_ce))]
    columns_1.append(col_to_add)
    col_to_add = ['['+  str(np.round(mean_squared_error(list_solutions_ce_spatial_01[ns][:,i], [true_params_effective_01_ce[i]]*100),3)) +']' for i in range(len(true_params_effective_01_ce))]
    columns_2.append(col_to_add)
    
for ns in range(3):
    col_to_add = [str(np.round(np.mean(list_solutions_ce_no_spatial_01[ns][:,i] - true_params_effective_no_spatial_ce[i]),3))
                  +' ('+str(np.round(np.std(list_solutions_ce_no_spatial_01[ns][:,i] - true_params_effective_no_spatial_ce[i]),3))+')' for i in range(len(true_params_effective_no_spatial_ce))] + ["/"]
    columns_1.append(col_to_add)
    col_to_add = [' ['+  str(np.round(mean_squared_error(list_solutions_ce_no_spatial_01[ns][:,i], [true_params_effective_no_spatial_ce[i]]*100),3)) +']' for i in range(len(true_params_effective_no_spatial_ce))] + ["/"]
    columns_2.append(col_to_add)
        
columns_1.insert(0,param_names)
columns_2.insert(0,param_names_2)

tr_columns_1 = np.transpose(columns_1)
tr_columns_2 = np.transpose(columns_2)
for i in range(len(tr_columns_1)):
    table_params.add_row(tr_columns_1[i])
    table_params.add_row(tr_columns_2[i])

print(table_params.draw())

+-----------+-----------+----------+----------+----------+----------+----------+
| Parameter | S n=50    | S n=200  | S n=1000 | NS n=50  | NS n=200 | NS       |
|           |           |          |          |          |          | n=1000   |
+-----------+-----------+----------+----------+----------+----------+----------+
| \multirow | 0.275     | 0.074    | 0.016    | 0.133    | -0.086   | -0.15    |
| {2}{*}{$\ | (0.054)   | (0.012)  | (0.006)  | (0.05)   | (0.023)  | (0.012)  |
| beta_{01} |           |          |          |          |          |          |
| $}        |           |          |          |          |          |          |
+-----------+-----------+----------+----------+----------+----------+----------+
|           | [0.079]   | [0.006]  | [0.0]    |  [0.02]  |  [0.008] |  [0.023] |
+-----------+-----------+----------+----------+----------+----------+----------+
| \multirow | -0.033    | -0.009   | -0.002   | 0.073    | 0.103    | 0.109    |
| {2}{*}{$\ | (0.027)   | (0

In [18]:
print(latextable.draw_latex(table_params, caption="Difference between the estimated parameters with the cross-entropy minimization and the true parameters ($rho=0.1$). The results are presented as the mean of the differences on the 100 iterations, the standard deviation within parenthesis, and the standard deviation within square brackets."))

\begin{table}
	\begin{center}
		\begin{tabular}{|l|l|l|l|l|l|l|}
			\hline
			 \\
			\hline
			Parameter & S n=50 & S n=200 & S n=1000 & NS n=50 & NS n=200 & NS n=1000 \\
			\hline
			\multirow{2}{*}{$\beta_{01}$} & 0.275 (0.054) & 0.074 (0.012) & 0.016 (0.006) & 0.133 (0.05) & -0.086 (0.023) & -0.15 (0.012) \\
			\hline
			 & [0.079] & [0.006] & [0.0] &  [0.02] &  [0.008] &  [0.023] \\
			\hline
			\multirow{2}{*}{$\beta_{02}$} & -0.033 (0.027) & -0.009 (0.006) & -0.002 (0.002) & 0.073 (0.037) & 0.103 (0.016) & 0.109 (0.007) \\
			\hline
			 & [0.002] & [0.0] & [0.0] &  [0.007] &  [0.011] &  [0.012] \\
			\hline
			\multirow{2}{*}{$\beta_{11}$} & -0.251 (0.052) & -0.068 (0.01) & -0.013 (0.005) & -0.247 (0.059) & -0.072 (0.021) & -0.018 (0.009) \\
			\hline
			 & [0.066] & [0.005] & [0.0] &  [0.064] &  [0.006] &  [0.0] \\
			\hline
			\multirow{2}{*}{$\beta_{12}$} & 0.117 (0.023) & 0.032 (0.008) & 0.007 (0.003) & 0.12 (0.027) & 0.032 (0.013) & 0.009 (0.005) \\
			\hline
			 & [0.014] &

In [19]:
table_params = Texttable()
table_params.add_row(["Parameter", "S n=50", "S n=200", "S n=1000",
                      "NS n=50", "NS n=200", "NS n=1000"])
                      
#param_names = ["$\\beta_{01}$", "", "$\\beta_{02}$", "", "$\\beta_{11}$", "", "$\\beta_{12}$", "",
#               "$\\beta_{21}$", "", "$\\beta_{22}$", "", "$\\gamma_0$", "", "$\\gamma_1$", "", "$\\rho$", ""]

param_names = ["\\multirow{2}{*}{$\\beta_{01}$}", "\\multirow{2}{*}{$\\beta_{02}$}", "\\multirow{2}{*}{$\\beta_{11}$}",
               "\\multirow{2}{*}{$\\beta_{12}$}", "\\multirow{2}{*}{$\\beta_{21}$}", "\\multirow{2}{*}{$\\beta_{22}$}",
               "\\multirow{2}{*}{$\\rho$}" ]
param_names_2 = [""]*len(param_names)

columns_1 = []
columns_2 = []

for ns in range(3):
    col_to_add = [str(np.round(np.mean(np.array(list_solutions_ce_spatial_05[ns])[:,i] - true_params_effective_05_ce[i]),3))
                  +' ('+str(np.round(np.std(np.array(list_solutions_ce_spatial_05[ns])[:,i] - true_params_effective_05_ce[i]),3))+')' for i in range(len(true_params_effective_05_ce))]
    columns_1.append(col_to_add)
    col_to_add = ['['+  str(np.round(mean_squared_error(np.array(list_solutions_ce_spatial_05[ns])[:,i], [true_params_effective_05_ce[i]]*len(list_solutions_ce_spatial_05[ns])),3)) +']' for i in range(len(true_params_effective_05_ce))]
    columns_2.append(col_to_add)
    
for ns in range(3):
    col_to_add = [str(np.round(np.mean(np.array(list_solutions_ce_no_spatial_05[ns])[:,i] - true_params_effective_no_spatial_ce[i]),3))
                  +' ('+str(np.round(np.std(np.array(list_solutions_ce_no_spatial_05[ns])[:,i] - true_params_effective_no_spatial_ce[i]),3))+')' for i in range(len(true_params_effective_no_spatial_ce))] + ["/"]
    columns_1.append(col_to_add)
    col_to_add = [' ['+  str(np.round(mean_squared_error(np.array(list_solutions_ce_no_spatial_05[ns])[:,i], [true_params_effective_no_spatial_ce[i]]*len(list_solutions_ce_no_spatial_05[ns])),3)) +']' for i in range(len(true_params_effective_no_spatial_ce))] + ["/"]
    columns_2.append(col_to_add)
        
columns_1.insert(0,param_names)
columns_2.insert(0,param_names_2)

tr_columns_1 = np.transpose(columns_1)
tr_columns_2 = np.transpose(columns_2)
for i in range(len(tr_columns_1)):
    table_params.add_row(tr_columns_1[i])
    table_params.add_row(tr_columns_2[i])

print(table_params.draw())

+-----------+-----------+----------+----------+----------+----------+----------+
| Parameter | S n=50    | S n=200  | S n=1000 | NS n=50  | NS n=200 | NS       |
|           |           |          |          |          |          | n=1000   |
+-----------+-----------+----------+----------+----------+----------+----------+
| \multirow | 0.436     | 0.133    | 0.026    | -0.399   | -0.801   | -0.9     |
| {2}{*}{$\ | (0.128)   | (0.029)  | (0.006)  | (0.238)  | (0.222)  | (0.101)  |
| beta_{01} |           |          |          |          |          |          |
| $}        |           |          |          |          |          |          |
+-----------+-----------+----------+----------+----------+----------+----------+
|           | [0.206]   | [0.019]  | [0.001]  |  [0.216] |  [0.691] |  [0.819] |
+-----------+-----------+----------+----------+----------+----------+----------+
| \multirow | -0.055    | -0.018   | -0.004   | 0.805    | 0.906    | 0.908    |
| {2}{*}{$\ | (0.052)   | (0

In [20]:
print(latextable.draw_latex(table_params, caption="Difference between the estimated parameters with the cross-entropy minimization and the true parameters ($rho=0.5$). The results are presented as the mean of the differences on the 100 iterations, the standard deviation within parenthesis, and the standard deviation within square brackets."))

\begin{table}
	\begin{center}
		\begin{tabular}{|l|l|l|l|l|l|l|}
			\hline
			 \\
			\hline
			Parameter & S n=50 & S n=200 & S n=1000 & NS n=50 & NS n=200 & NS n=1000 \\
			\hline
			\multirow{2}{*}{$\beta_{01}$} & 0.436 (0.128) & 0.133 (0.029) & 0.026 (0.006) & -0.399 (0.238) & -0.801 (0.222) & -0.9 (0.101) \\
			\hline
			 & [0.206] & [0.019] & [0.001] &  [0.216] &  [0.691] &  [0.819] \\
			\hline
			\multirow{2}{*}{$\beta_{02}$} & -0.055 (0.052) & -0.018 (0.008) & -0.004 (0.002) & 0.805 (0.239) & 0.906 (0.123) & 0.908 (0.055) \\
			\hline
			 & [0.006] & [0.0] & [0.0] &  [0.705] &  [0.836] &  [0.827] \\
			\hline
			\multirow{2}{*}{$\beta_{11}$} & -0.486 (0.19) & -0.141 (0.034) & -0.027 (0.006) & -0.512 (0.254) & -0.316 (0.136) & -0.246 (0.059) \\
			\hline
			 & [0.273] & [0.021] & [0.001] &  [0.327] &  [0.118] &  [0.064] \\
			\hline
			\multirow{2}{*}{$\beta_{12}$} & 0.162 (0.039) & 0.047 (0.008) & 0.01 (0.003) & 0.191 (0.092) & 0.095 (0.062) & 0.071 (0.024) \\
			\hline
			 & [

In [21]:
table_params = Texttable()
table_params.add_row(["Parameter", "S n=50", "S n=200", "S n=1000",
                      "NS n=50", "NS n=200", "NS n=1000"])
                      
#param_names = ["$\\beta_{01}$", "", "$\\beta_{02}$", "", "$\\beta_{11}$", "", "$\\beta_{12}$", "",
#               "$\\beta_{21}$", "", "$\\beta_{22}$", "", "$\\gamma_0$", "", "$\\gamma_1$", "", "$\\rho$", ""]

param_names = ["\\multirow{2}{*}{$\\beta_{01}$}", "\\multirow{2}{*}{$\\beta_{02}$}", "\\multirow{2}{*}{$\\beta_{11}$}",
               "\\multirow{2}{*}{$\\beta_{12}$}", "\\multirow{2}{*}{$\\beta_{21}$}", "\\multirow{2}{*}{$\\beta_{22}$}",
               "\\multirow{2}{*}{$\\rho$}" ]
param_names_2 = [""]*len(param_names)

columns_1 = []
columns_2 = []

for ns in range(3):
    col_to_add = [str(np.round(np.mean(np.array(list_solutions_ce_spatial_09[ns])[:,i] - true_params_effective_09_ce[i]),3))
                  +' ('+str(np.round(np.std(np.array(list_solutions_ce_spatial_09[ns])[:,i] - true_params_effective_09_ce[i]),3))+')' for i in range(len(true_params_effective_09_ce))]
    columns_1.append(col_to_add)
    col_to_add = ['['+  str(np.round(mean_squared_error(np.array(list_solutions_ce_spatial_09[ns])[:,i], [true_params_effective_09_ce[i]]*len(list_solutions_ce_spatial_09[ns])),3)) +']' for i in range(len(true_params_effective_09_ce))]
    columns_2.append(col_to_add)
    
for ns in range(3):
    col_to_add = [str(np.round(np.mean(np.array(list_solutions_ce_no_spatial_09[ns])[:,i] - true_params_effective_no_spatial_ce[i]),3))
                  +' ('+str(np.round(np.std(np.array(list_solutions_ce_no_spatial_09[ns])[:,i] - true_params_effective_no_spatial_ce[i]),3))+')' for i in range(len(true_params_effective_no_spatial_ce))] + ["/"]
    columns_1.append(col_to_add)
    col_to_add = [' ['+  str(np.round(mean_squared_error(np.array(list_solutions_ce_no_spatial_09[ns])[:,i], [true_params_effective_no_spatial_ce[i]]*len(list_solutions_ce_no_spatial_09[ns])),3)) +']' for i in range(len(true_params_effective_no_spatial_ce))] + ["/"]
    columns_2.append(col_to_add)
        
columns_1.insert(0,param_names)
columns_2.insert(0,param_names_2)

tr_columns_1 = np.transpose(columns_1)
tr_columns_2 = np.transpose(columns_2)
for i in range(len(tr_columns_1)):
    table_params.add_row(tr_columns_1[i])
    table_params.add_row(tr_columns_2[i])

print(table_params.draw())

+-----------+-----------+----------+----------+----------+----------+----------+
| Parameter | S n=50    | S n=200  | S n=1000 | NS n=50  | NS n=200 | NS       |
|           |           |          |          |          |          | n=1000   |
+-----------+-----------+----------+----------+----------+----------+----------+
| \multirow | 1.53      | 1.522    | 1.128    | 1.091    | 0.648    | -0.415   |
| {2}{*}{$\ | (0.227)   | (0.237)  | (0.346)  | (0.742)  | (0.877)  | (0.801)  |
| beta_{01} |           |          |          |          |          |          |
| $}        |           |          |          |          |          |          |
+-----------+-----------+----------+----------+----------+----------+----------+
|           | [2.394]   | [2.372]  | [1.392]  |  [1.74]  |  [1.189] |  [0.813] |
+-----------+-----------+----------+----------+----------+----------+----------+
| \multirow | 0.439     | -0.076   | -0.115   | 3.438    | 4.328    | 4.324    |
| {2}{*}{$\ | (0.846)   | (0

In [22]:
print(latextable.draw_latex(table_params, caption="Difference between the estimated parameters with the cross-entropy minimization and the true parameters ($rho=0.9$). The results are presented as the mean of the differences on the 100 iterations, the standard deviation within parenthesis, and the standard deviation within square brackets."))

\begin{table}
	\begin{center}
		\begin{tabular}{|l|l|l|l|l|l|l|}
			\hline
			 \\
			\hline
			Parameter & S n=50 & S n=200 & S n=1000 & NS n=50 & NS n=200 & NS n=1000 \\
			\hline
			\multirow{2}{*}{$\beta_{01}$} & 1.53 (0.227) & 1.522 (0.237) & 1.128 (0.346) & 1.091 (0.742) & 0.648 (0.877) & -0.415 (0.801) \\
			\hline
			 & [2.394] & [2.372] & [1.392] &  [1.74] &  [1.189] &  [0.813] \\
			\hline
			\multirow{2}{*}{$\beta_{02}$} & 0.439 (0.846) & -0.076 (0.357) & -0.115 (0.043) & 3.438 (0.768) & 4.328 (0.907) & 4.324 (0.717) \\
			\hline
			 & [0.909] & [0.133] & [0.015] &  [12.407] &  [19.557] &  [19.208] \\
			\hline
			\multirow{2}{*}{$\beta_{11}$} & -1.994 (0.435) & -2.2 (0.508) & -1.499 (0.658) & -2.003 (0.574) & -2.226 (0.634) & -1.669 (0.521) \\
			\hline
			 & [4.165] & [5.097] & [2.68] &  [4.342] &  [5.357] &  [3.059] \\
			\hline
			\multirow{2}{*}{$\beta_{12}$} & 0.914 (0.445) & 0.566 (0.282) & 0.217 (0.06) & 0.841 (0.523) & 0.464 (0.42) & 0.341 (0.207) \\
			\hline
			 & 

# Analysis of the results on a test set

In [16]:
def analysis_results(list_solutions_spatial, list_solutions_no_spatial, rho,
                     n_repeat=100, size_test=1000, list_n_samples=[50,200,1000]):
    
    list_r2_s, list_rmse_s, list_similarity_s, list_crossentropy_s = [], [], [], []
    list_r2_ns, list_rmse_ns, list_similarity_ns, list_crossentropy_ns = [], [], [], []

    seed=0

    for n_samples_index in range(3):
        n_samples = list_n_samples[n_samples_index]
        
        true_params = np.concatenate([beta.flatten(),gamma_var, [rho]])
        
        list_n_i = np.random.randint(100, high=10000, size=n_samples)
        
        list_r2_s_temp, list_rmse_s_temp, list_similarity_s_temp, list_crossentropy_s_temp = [], [], [], []
        list_r2_ns_temp, list_rmse_ns_temp, list_similarity_ns_temp, list_crossentropy_ns_temp = [], [], [], []
        
        for i in range(len(list_solutions_spatial[n_samples_index])):
            np.random.seed(seed+1000)

            X,Z,W = create_features_matrices(n_samples,n_features,choice_W='random_distance',nneighbors=10,cov_mat=cov_matrix)
            Z[:,0] = 1
            
            #list_n_i = np.random.randint(100, high=10000, size=n_samples)
            M = np.identity(n_samples) - rho*W
            mu = dirichlet_regression.compute_mu_spatial(X, beta, M)
            try:
                
                Y = np.array([np.random.multinomial(list_n_i[i],mu[i])/list_n_i[i] for i in range(n_samples)])
                Y = (Y*(n_samples-1)+1/n_classes)/n_samples

                solution_spatial = list_solutions_spatial[n_samples_index][i]
                beta_sol_s = np.zeros((n_features+1,n_classes))
                beta_sol_s[:,1:] = solution_spatial[:(n_features+1)*(n_classes-1)].reshape((n_features+1),n_classes-1)
                rho_sol_s = solution_spatial[-1]
                M_sol = np.identity(n_samples) - rho_sol_s*W
                mu_sol_s = dirichlet_regression.compute_mu_spatial(X, beta_sol_s, M_sol)

                list_r2_s_temp.append(r2_score(Y,mu_sol_s))
                list_rmse_s_temp.append(mean_squared_error(Y,mu_sol_s,squared=False))
                list_crossentropy_s_temp.append(-(1/size_test)*np.sum(Y*np.log(mu_sol_s)))
                list_similarity_s_temp.append(cos_similarity(Y, mu_sol_s))

                solution_no_spatial = list_solutions_no_spatial[n_samples_index][i]
                beta_sol_ns = np.zeros((n_features+1,n_classes))
                beta_sol_ns[:,1:] = solution_no_spatial[:(n_features+1)*(n_classes-1)].reshape((n_features+1),n_classes-1)
                mu_sol_ns = dirichlet_regression.compute_mu(X, beta_sol_ns)

                list_r2_ns_temp.append(r2_score(Y,mu_sol_ns))
                list_rmse_ns_temp.append(mean_squared_error(Y,mu_sol_ns,squared=False))
                list_crossentropy_ns_temp.append(-(1/size_test)*np.sum(Y*np.log(mu_sol_ns)))
                list_similarity_ns_temp.append(cos_similarity(Y, mu_sol_ns))

            except RuntimeError:
                print("Factor is exactly singular")
            except np.linalg.LinAlgError:
                print("Singular matrix")

            seed+=1
        list_r2_s.append(list_r2_s_temp)
        list_rmse_s.append(list_rmse_s_temp)
        list_similarity_s.append(list_similarity_s_temp)
        list_crossentropy_s.append(list_crossentropy_s_temp)
        list_r2_ns.append(list_r2_ns_temp)
        list_rmse_ns.append(list_rmse_ns_temp)
        list_similarity_ns.append(list_similarity_ns_temp)
        list_crossentropy_ns.append(list_crossentropy_ns_temp)
    return(list_r2_s, list_rmse_s, list_similarity_s, list_crossentropy_s,
           list_r2_ns, list_rmse_ns, list_similarity_ns, list_crossentropy_ns)

In [17]:
def details_r2_mu(list_solutions_spatial, list_solutions_no_spatial, rho,
                     n_repeat=100, size_test=1000, list_n_samples=[50,200,1000]):
    
    list_r2_s, list_r2_ns = [], []
    list_mu_s, list_mu_ns = [], []
    list_true_mu = []

    seed=0

    for n_samples_index in range(3):
        n_samples = list_n_samples[n_samples_index]
        
        true_params = np.concatenate([beta.flatten(),gamma_var, [rho]])
        
        list_n_i = np.random.randint(100, high=10000, size=n_samples)
        
        list_r2_s_temp, list_r2_ns_temp = [], []
        list_mu_s_temp, list_mu_ns_temp = [], []
        list_true_mu_temp = []
        
        for i in range(len(list_solutions_spatial[n_samples_index])):
            np.random.seed(seed+1000)

            X,Z,W = create_features_matrices(n_samples,n_features,choice_W='random_distance',nneighbors=10,cov_mat=cov_matrix)
            Z[:,0] = 1
            
            #list_n_i = np.random.randint(100, high=10000, size=n_samples)
            M = np.identity(n_samples) - rho*W
            mu = dirichlet_regression.compute_mu_spatial(X, beta, M)
            try:
                
                Y = np.array([np.random.multinomial(list_n_i[i],mu[i])/list_n_i[i] for i in range(n_samples)])
                Y = (Y*(n_samples-1)+1/n_classes)/n_samples
                
                list_true_mu_temp.append(Y)

                solution_spatial = list_solutions_spatial[n_samples_index][i]
                beta_sol_s = np.zeros((n_features+1,n_classes))
                beta_sol_s[:,1:] = solution_spatial[:(n_features+1)*(n_classes-1)].reshape((n_features+1),n_classes-1)
                rho_sol_s = solution_spatial[-1]
                M_sol = np.identity(n_samples) - rho_sol_s*W
                mu_sol_s = dirichlet_regression.compute_mu_spatial(X, beta_sol_s, M_sol)

                list_mu_s_temp.append(mu_sol_s)
                list_r2_s_temp.append(r2_score(Y,mu_sol_s, multioutput='raw_values'))

                solution_no_spatial = list_solutions_no_spatial[n_samples_index][i]
                beta_sol_ns = np.zeros((n_features+1,n_classes))
                beta_sol_ns[:,1:] = solution_no_spatial[:(n_features+1)*(n_classes-1)].reshape((n_features+1),n_classes-1)
                mu_sol_ns = dirichlet_regression.compute_mu(X, beta_sol_ns)

                list_mu_ns_temp.append(mu_sol_ns)
                list_r2_ns_temp.append(r2_score(Y,mu_sol_ns, multioutput='raw_values'))

            except RuntimeError:
                print("Factor is exactly singular")
            except np.linalg.LinAlgError:
                print("Singular matrix")

            seed+=1
        list_true_mu.append(list_true_mu_temp)
        list_mu_s.append(list_mu_s_temp)
        list_mu_ns.append(list_mu_ns_temp)
        list_r2_s.append(list_r2_s_temp)
        list_r2_ns.append(list_r2_ns_temp)
    return(list_r2_s, list_r2_ns, list_true_mu, list_mu_s, list_mu_ns)

In [57]:
rho=0.5

In [59]:
seed=0

for n_samples_index in range(1):
    n_samples = list_n_samples[n_samples_index]

    true_params = np.concatenate([beta.flatten(),gamma_var, [rho]])

    list_n_i = np.random.randint(100, high=10000, size=n_samples)

    for i in range(10):
        np.random.seed(seed+1000)

        X,Z,W = create_features_matrices(n_samples,n_features,choice_W='random_distance',nneighbors=10,cov_mat=cov_matrix)
        Z[:,0] = 1

        #list_n_i = np.random.randint(100, high=10000, size=n_samples)
        M = np.identity(n_samples) - rho*W
        mu = dirichlet_regression.compute_mu_spatial(X, beta, M)
        try:

            #print(mu[i])
            Y = np.array([np.random.multinomial(list_n_i[i],mu[i])/list_n_i[i] for i in range(n_samples)])
            print(Y)
            Y = (Y*(n_samples-1)+1/n_classes)/n_samples

        except RuntimeError:
            print("Factor is exactly singular")
        except np.linalg.LinAlgError:
            print("Singular matrix")

        seed+=1

[[3.30115830e-01 6.94980695e-02 6.00386100e-01]
 [2.69670663e-01 3.92724266e-02 6.91056911e-01]
 [2.07871835e-01 1.89873418e-02 7.73140823e-01]
 [1.89418778e-01 1.90760060e-02 7.91505216e-01]
 [3.73467113e-02 0.00000000e+00 9.62653289e-01]
 [4.74337595e-02 7.41152492e-04 9.51825088e-01]
 [2.69378949e-01 6.46786870e-01 8.38341809e-02]
 [8.76944837e-02 2.35737860e-03 9.09948138e-01]
 [1.58545727e-01 1.27436282e-02 8.28710645e-01]
 [7.42942051e-03 0.00000000e+00 9.92570579e-01]
 [1.96771078e-01 1.83524217e-02 7.84876501e-01]
 [4.04458026e-02 0.00000000e+00 9.59554197e-01]
 [3.26820191e-02 4.65224471e-04 9.66852756e-01]
 [3.93186813e-01 3.30109890e-01 2.76703297e-01]
 [3.13791649e-01 7.14185295e-02 6.14789821e-01]
 [1.18329466e-01 1.16009281e-03 8.80510441e-01]
 [3.14759743e-01 7.87290656e-02 6.06511191e-01]
 [1.33311604e-01 4.88917862e-03 8.61799218e-01]
 [3.27073552e-01 8.05946792e-02 5.92331768e-01]
 [3.02141158e-01 5.90800952e-01 1.07057891e-01]
 [3.89293139e-01 3.30691268e-01 2.800155

## rho=0.1

In [26]:
%%time
list_r2_s, list_rmse_s, list_similarity_s, list_crossentropy_s, list_r2_ns, list_rmse_ns, list_similarity_ns, list_crossentropy_ns = analysis_results(list_solutions_spatial_01, list_solutions_no_spatial_01, rho=0.1) 

Wall time: 9.52 s


In [27]:
print("DIRICHLET no spatial")
print('R2', np.round(np.mean(list_r2_ns[2]),4), '(', np.round(np.var(list_r2_ns[2]),4), ')')
print('RMSE', np.round(np.mean(list_rmse_ns[2]),4), '(', np.round(np.var(list_rmse_ns[2]),4), ')')
print('Crossentropy', np.round(np.mean(list_crossentropy_ns[2]),4), '(', np.round(np.var(list_crossentropy_ns[2]),4), ')')
print('Cos similarity', np.round(np.mean(list_similarity_ns[2]),4), '(', np.round(np.var(list_similarity_ns[2]),4), ')')
print("---\nDIRICHLET spatial")
print('R2', np.round(np.mean(list_r2_s[2]),4), '(', np.round(np.var(list_r2_s[2]),4), ')')
print('RMSE', np.round(np.mean(list_rmse_s[2]),4), '(', np.round(np.var(list_rmse_s[2]),4), ')')
print('Crossentropy', np.round(np.mean(list_crossentropy_s[2]),4), '(', np.round(np.var(list_crossentropy_s[2]),4), ')')
print('Cos similarity', np.round(np.mean(list_similarity_s[2]),4), '(', np.round(np.var(list_similarity_s[2]),4), ')')

DIRICHLET no spatial
R2 0.9947 ( 0.0 )
RMSE 0.0159 ( 0.0 )
Crossentropy 0.5752 ( 0.0001 )
Cos similarity 0.9992 ( 0.0 )
---
DIRICHLET spatial
R2 0.9976 ( 0.0 )
RMSE 0.0078 ( 0.0 )
Crossentropy 0.5742 ( 0.0001 )
Cos similarity 0.9998 ( 0.0 )


In [28]:
%%time
list_r2_s, list_rmse_s, list_similarity_s, list_crossentropy_s, list_r2_ns, list_rmse_ns, list_similarity_ns, list_crossentropy_ns = analysis_results(list_solutions_ce_spatial_01, list_solutions_ce_no_spatial_01, rho=0.1)

Wall time: 8.94 s


In [29]:
print("CROSSENTROPY no spatial")
print('R2', np.round(np.mean(list_r2_ns[2]),4), '(', np.round(np.var(list_r2_ns[2]),4), ')')
print('RMSE', np.round(np.mean(list_rmse_ns[2]),4), '(', np.round(np.var(list_rmse_ns[2]),4), ')')
print('Crossentropy', np.round(np.mean(list_crossentropy_ns[2]),4), '(', np.round(np.var(list_crossentropy_ns[2]),4), ')')
print('Cos similarity', np.round(np.mean(list_similarity_ns[2]),4), '(', np.round(np.var(list_similarity_ns[2]),4), ')')
print("---\nCROSSENTROPY spatial")
print('R2', np.round(np.mean(list_r2_s[2]),4), '(', np.round(np.var(list_r2_s[2]),4), ')')
print('RMSE', np.round(np.mean(list_rmse_s[2]),4), '(', np.round(np.var(list_rmse_s[2]),4), ')')
print('Crossentropy', np.round(np.mean(list_crossentropy_s[2]),4), '(', np.round(np.var(list_crossentropy_s[2]),4), ')')
print('Cos similarity', np.round(np.mean(list_similarity_s[2]),4), '(', np.round(np.var(list_similarity_s[2]),4), ')')

CROSSENTROPY no spatial
R2 0.9948 ( 0.0 )
RMSE 0.0158 ( 0.0 )
Crossentropy 0.5752 ( 0.0001 )
Cos similarity 0.9992 ( 0.0 )
---
CROSSENTROPY spatial
R2 0.9984 ( 0.0 )
RMSE 0.007 ( 0.0 )
Crossentropy 0.574 ( 0.0001 )
Cos similarity 0.9999 ( 0.0 )


## rho=0.5

In [30]:
%%time
list_r2_s, list_rmse_s, list_similarity_s, list_crossentropy_s, list_r2_ns, list_rmse_ns, list_similarity_ns, list_crossentropy_ns = analysis_results(list_solutions_spatial_05, list_solutions_no_spatial_05, rho=0.5) 

Wall time: 9.05 s


In [31]:
print("DIRICHLET no spatial")
print('R2', np.round(np.mean(list_r2_ns[2]),4), '(', np.round(np.var(list_r2_ns[2]),4), ')')
print('RMSE', np.round(np.mean(list_rmse_ns[2]),4), '(', np.round(np.var(list_rmse_ns[2]),4), ')')
print('Crossentropy', np.round(np.mean(list_crossentropy_ns[2]),4), '(', np.round(np.var(list_crossentropy_ns[2]),4), ')')
print('Cos similarity', np.round(np.mean(list_similarity_ns[2]),4), '(', np.round(np.var(list_similarity_ns[2]),4), ')')
print("---\nDIRICHLET spatial")
print('R2', np.round(np.mean(list_r2_s[2]),4), '(', np.round(np.var(list_r2_s[2]),4), ')')
print('RMSE', np.round(np.mean(list_rmse_s[2]),4), '(', np.round(np.var(list_rmse_s[2]),4), ')')
print('Crossentropy', np.round(np.mean(list_crossentropy_s[2]),4), '(', np.round(np.var(list_crossentropy_s[2]),4), ')')
print('Cos similarity', np.round(np.mean(list_similarity_s[2]),4), '(', np.round(np.var(list_similarity_s[2]),4), ')')

DIRICHLET no spatial
R2 0.8096 ( 0.0002 )
RMSE 0.0919 ( 0.0 )
Crossentropy 0.5086 ( 0.0003 )
Cos similarity 0.9789 ( 0.0 )
---
DIRICHLET spatial
R2 0.9974 ( 0.0001 )
RMSE 0.0071 ( 0.0 )
Crossentropy 0.4527 ( 0.0003 )
Cos similarity 0.9998 ( 0.0 )


In [32]:
list_r2_s, list_rmse_s, list_similarity_s, list_crossentropy_s, list_r2_ns, list_rmse_ns, list_similarity_ns, list_crossentropy_ns = analysis_results(list_solutions_ce_spatial_05, list_solutions_ce_no_spatial_05, rho=0.5)

In [33]:
print("CROSSENTROPY no spatial")
print('R2', np.round(np.mean(list_r2_ns[2]),4), '(', np.round(np.var(list_r2_ns[2]),4), ')')
print('RMSE', np.round(np.mean(list_rmse_ns[2]),4), '(', np.round(np.var(list_rmse_ns[2]),4), ')')
print('Crossentropy', np.round(np.mean(list_crossentropy_ns[2]),4), '(', np.round(np.var(list_crossentropy_ns[2]),4), ')')
print('Cos similarity', np.round(np.mean(list_similarity_ns[2]),4), '(', np.round(np.var(list_similarity_ns[2]),4), ')')
print("---\nCROSSENTROPY spatial")
print('R2', np.round(np.mean(list_r2_s[2]),4), '(', np.round(np.var(list_r2_s[2]),4), ')')
print('RMSE', np.round(np.mean(list_rmse_s[2]),4), '(', np.round(np.var(list_rmse_s[2]),4), ')')
print('Crossentropy', np.round(np.mean(list_crossentropy_s[2]),4), '(', np.round(np.var(list_crossentropy_s[2]),4), ')')
print('Cos similarity', np.round(np.mean(list_similarity_s[2]),4), '(', np.round(np.var(list_similarity_s[2]),4), ')')

CROSSENTROPY no spatial
R2 0.8612 ( 0.0002 )
RMSE 0.0795 ( 0.0 )
Crossentropy 0.4893 ( 0.0003 )
Cos similarity 0.9819 ( 0.0 )
---
CROSSENTROPY spatial
R2 0.9987 ( 0.0 )
RMSE 0.0062 ( 0.0 )
Crossentropy 0.4522 ( 0.0003 )
Cos similarity 0.9999 ( 0.0 )


## rho=0.9

In [19]:
%%time
list_r2_s2, list_r2_ns2, list_true_mu, list_mu_s, list_mu_ns = details_r2_mu(list_solutions_spatial_09, list_solutions_no_spatial_09, rho=0.9) 

Wall time: 41.4 s


In [44]:
list_r2_s2[2][19]

array([-3.54565125e+00, -1.08523137e+04, -1.22899813e+01])

In [51]:
np.var(list_true_mu[2][19],axis=0)

array([3.63419112e-04, 9.08706748e-08, 3.72215104e-04])

In [29]:
np.mean(list_true_mu[1][0], axis=0)

array([0.00409177, 0.00166667, 0.99424156])

In [30]:
list_true_mu[1][0]

array([[0.0048693 , 0.00166667, 0.99346403],
       [0.00166667, 0.00166667, 0.99666667],
       [0.00199665, 0.00166667, 0.99633669],
       [0.00166667, 0.00166667, 0.99666667],
       [0.00203423, 0.00166667, 0.9962991 ],
       [0.00166667, 0.00166667, 0.99666667],
       [0.00196395, 0.00166667, 0.99636939],
       [0.00166667, 0.00166667, 0.99666667],
       [0.00166667, 0.00166667, 0.99666667],
       [0.00166667, 0.00166667, 0.99666667],
       [0.00445709, 0.00166667, 0.99387624],
       [0.00166667, 0.00166667, 0.99666667],
       [0.00210345, 0.00166667, 0.99622988],
       [0.00166667, 0.00166667, 0.99666667],
       [0.00166667, 0.00166667, 0.99666667],
       [0.00166667, 0.00166667, 0.99666667],
       [0.00166667, 0.00166667, 0.99666667],
       [0.00166667, 0.00166667, 0.99666667],
       [0.00166667, 0.00166667, 0.99666667],
       [0.00192551, 0.00166667, 0.99640782],
       [0.00166667, 0.00166667, 0.99666667],
       [0.00207743, 0.00166667, 0.99625591],
       [0.

In [23]:
np.mean(list_true_mu[1],axis=0)

array([[0.01347565, 0.0107505 , 0.97577385],
       [0.02100267, 0.00290839, 0.97608894],
       [0.02333426, 0.00891413, 0.96775161],
       [0.01910317, 0.0018654 , 0.97903143],
       [0.01773296, 0.00288331, 0.97938373],
       [0.01134714, 0.00177366, 0.98687921],
       [0.02883519, 0.00379817, 0.96736664],
       [0.02185008, 0.00208233, 0.97606759],
       [0.02498921, 0.00540055, 0.96961024],
       [0.01326263, 0.00249097, 0.98424639],
       [0.01124211, 0.00245764, 0.98630025],
       [0.01047661, 0.01165066, 0.97787273],
       [0.025291  , 0.01149582, 0.96321318],
       [0.01349929, 0.00313346, 0.98336725],
       [0.01511773, 0.01151815, 0.97336412],
       [0.00486949, 0.00167426, 0.99345625],
       [0.01465916, 0.00253373, 0.98280711],
       [0.02328272, 0.00351385, 0.97320342],
       [0.00792359, 0.00168512, 0.99039129],
       [0.01366672, 0.00590137, 0.98043191],
       [0.01696595, 0.01173021, 0.97130384],
       [0.01182889, 0.00196241, 0.98620869],
       [0.

In [52]:
list_true_mu[2][19][:,1]-np.mean(list_true_mu[2][19], axis=0)[1]

array([-1.38256403e-05, -1.38256403e-05, -1.38256403e-05, -1.38256403e-05,
       -1.38256403e-05, -1.38256403e-05, -1.38256403e-05, -1.38256403e-05,
       -1.38256403e-05, -1.38256403e-05, -1.38256403e-05, -1.38256403e-05,
       -1.38256403e-05, -1.38256403e-05, -1.38256403e-05, -1.38256403e-05,
       -1.38256403e-05, -1.38256403e-05, -1.38256403e-05, -1.38256403e-05,
       -1.38256403e-05, -1.38256403e-05, -1.38256403e-05, -1.38256403e-05,
       -1.38256403e-05, -1.38256403e-05, -1.38256403e-05, -1.38256403e-05,
       -1.38256403e-05, -1.38256403e-05, -1.38256403e-05, -1.38256403e-05,
       -1.38256403e-05, -1.38256403e-05, -1.38256403e-05, -1.38256403e-05,
       -1.38256403e-05, -1.38256403e-05, -1.38256403e-05, -1.38256403e-05,
       -1.38256403e-05, -1.38256403e-05, -1.38256403e-05, -1.38256403e-05,
       -1.38256403e-05, -1.38256403e-05, -1.38256403e-05, -1.38256403e-05,
       -1.38256403e-05, -1.38256403e-05, -1.38256403e-05, -1.38256403e-05,
       -1.38256403e-05, -

In [46]:
list_mu_s[2][19]

array([[0.04914457, 0.03477732, 0.91607811],
       [0.04453226, 0.03331746, 0.92215029],
       [0.01959592, 0.02871789, 0.95168619],
       ...,
       [0.00437241, 0.01812696, 0.97750062],
       [0.04184338, 0.03370933, 0.92444729],
       [0.02706112, 0.03074872, 0.94219015]])

In [18]:
%%time
list_r2_s, list_rmse_s, list_similarity_s, list_crossentropy_s, list_r2_ns, list_rmse_ns, list_similarity_ns, list_crossentropy_ns = analysis_results(list_solutions_spatial_09, list_solutions_no_spatial_09, rho=0.9) 

Wall time: 12.5 s


In [26]:
np.mean(list_r2_s[0])

-1.4549499311203825e+29

In [27]:
np.mean(list_r2_s[1])

-1.4349470128665635e+31

In [28]:
np.mean(list_r2_s[2])

-58.232483605315025

In [35]:
print("DIRICHLET no spatial")
print('R2', np.round(np.mean(list_r2_ns[2]),4), '(', np.round(np.var(list_r2_ns[2]),4), ')')
print('RMSE', np.round(np.mean(list_rmse_ns[2]),4), '(', np.round(np.var(list_rmse_ns[2]),4), ')')
print('Crossentropy', np.round(np.mean(list_crossentropy_ns[2]),4), '(', np.round(np.var(list_crossentropy_ns[2]),4), ')')
print('Cos similarity', np.round(np.mean(list_similarity_ns[2]),4), '(', np.round(np.var(list_similarity_ns[2]),4), ')')
print("---\nDIRICHLET spatial")
print('R2', np.round(np.mean(list_r2_s[2]),4), '(', np.round(np.var(list_r2_s[2]),4), ')')
print('RMSE', np.round(np.mean(list_rmse_s[2]),4), '(', np.round(np.var(list_rmse_s[2]),4), ')')
print('Crossentropy', np.round(np.mean(list_crossentropy_s[2]),4), '(', np.round(np.var(list_crossentropy_s[2]),4), ')')
print('Cos similarity', np.round(np.mean(list_similarity_s[2]),4), '(', np.round(np.var(list_similarity_s[2]),4), ')')

DIRICHLET no spatial
R2 -122.2414 ( 634339.985 )
RMSE 0.072 ( 0.0006 )
Crossentropy 0.1162 ( 0.0018 )
Cos similarity 0.9892 ( 0.0 )
---
DIRICHLET spatial
R2 -63.792 ( 164230.4716 )
RMSE 0.0609 ( 0.0006 )
Crossentropy 0.0938 ( 0.0012 )
Cos similarity 0.9912 ( 0.0 )


In [36]:
list_r2_s, list_rmse_s, list_similarity_s, list_crossentropy_s, list_r2_ns, list_rmse_ns, list_similarity_ns, list_crossentropy_ns = analysis_results(list_solutions_ce_spatial_09, list_solutions_ce_no_spatial_09, rho=0.9) 

In [37]:
print("CROSSENTROPY no spatial")
print('R2', np.round(np.mean(list_r2_ns[2]),4), '(', np.round(np.var(list_r2_ns[2]),4), ')')
print('RMSE', np.round(np.mean(list_rmse_ns[2]),4), '(', np.round(np.var(list_rmse_ns[2]),4), ')')
print('Crossentropy', np.round(np.mean(list_crossentropy_ns[2]),4), '(', np.round(np.var(list_crossentropy_ns[2]),4), ')')
print('Cos similarity', np.round(np.mean(list_similarity_ns[2]),4), '(', np.round(np.var(list_similarity_ns[2]),4), ')')
print("---\nCROSSENTROPY spatial")
print('R2', np.round(np.mean(list_r2_s[2]),4), '(', np.round(np.var(list_r2_s[2]),4), ')')
print('RMSE', np.round(np.mean(list_rmse_s[2]),4), '(', np.round(np.var(list_rmse_s[2]),4), ')')
print('Crossentropy', np.round(np.mean(list_crossentropy_s[2]),4), '(', np.round(np.var(list_crossentropy_s[2]),4), ')')
print('Cos similarity', np.round(np.mean(list_similarity_s[2]),4), '(', np.round(np.var(list_similarity_s[2]),4), ')')

CROSSENTROPY no spatial
R2 -53.5842 ( 117367.7133 )
RMSE 0.0665 ( 0.0005 )
Crossentropy 0.0938 ( 0.0017 )
Cos similarity 0.9899 ( 0.0 )
---
CROSSENTROPY spatial
R2 0.1786 ( 12.1084 )
RMSE 0.0208 ( 0.0003 )
Crossentropy 0.052 ( 0.0005 )
Cos similarity 0.9983 ( 0.0 )
